In [53]:
import os
import json
from opensearchpy import OpenSearch, RequestsHttpConnection, helpers
import boto3
from requests_aws4auth import AWS4Auth
from config import *

In [54]:
# ─── Configuration ──────────────────────────────────────────────────────────────
DOMAIN = "search-redcoat-express-wvba3hxtx72luhtzgh7tb56k4i.aos.eu-north-1.on.aws"
REGION = "eu-north-1"
INDEX  = "redcoat-express"
MASTER_USER        = "admin"              # the internal username you configured
MASTER_PASSWORD    = "ASINnumber1!" # the password you set

DATA_DIRS = [email_chunks_dir, attachment_chunks_dir, thread_documents_dir]

Access over IAM (Switched Off)

In [ ]:
# # Creating a session for a specific local user "rag-admin-user"
# session = boto3.Session(profile_name="rag-admin-user")

# # Accessing STS to assume the role with the right permissions"
# sts = session.client("sts")
# response = sts.assume_role(
#     RoleArn="arn:aws:iam::169189304433:role/RedcoatExpressRagAdmin",
#     RoleSessionName="OpenSearchUpload"
# )

# # Extracting credentials from the session
# credentials = response["Credentials"]

# # Passing credentials to AWSAUTH so we can authenticate to OpenSearch
# awsauth = AWS4Auth(
#     credentials["AccessKeyId"],
#     credentials["SecretAccessKey"],
#     REGION,
#     "es",
#     session_token=credentials["SessionToken"]
# )

# # Creating OpenSearch client with A.) my created OpenSearch domain and B.) my AWSAUTH credentials
# client = OpenSearch(
#     hosts=[{"host": DOMAIN, "port": 443}],
#     http_auth=awsauth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )

# # Testing connection:
# print(client.cat.indices(format="json"))

Access over OpenSearch Master User

In [55]:

client = OpenSearch(
    hosts=[{"host": DOMAIN, "port": 443}],
    http_auth=(MASTER_USER, MASTER_PASSWORD),
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

# Test it:
print(client.cat.indices(format="json"))

[{'health': 'green', 'status': 'open', 'index': 'otel-v1-apm-service-map-sample', 'uuid': 'InKqxx9zRB-xtgzA46VoUQ', 'pri': '1', 'rep': '2', 'docs.count': '49', 'docs.deleted': '0', 'store.size': '52.9kb', 'pri.store.size': '17.6kb'}, {'health': 'green', 'status': 'open', 'index': '.plugins-ml-model-group', 'uuid': 'GK4aZPJnTUGosjC69CfIyQ', 'pri': '1', 'rep': '2', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '11.2kb', 'pri.store.size': '228b'}, {'health': 'green', 'status': 'open', 'index': '.opensearch-observability', 'uuid': '45kj-BwKTZq0vUu1ps6F3w', 'pri': '1', 'rep': '2', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '624b', 'pri.store.size': '208b'}, {'health': 'green', 'status': 'open', 'index': '.plugins-ml-config', 'uuid': 'Jk7L7STERNqoQw-xVoENRw', 'pri': '1', 'rep': '2', 'docs.count': '1', 'docs.deleted': '0', 'store.size': '11.8kb', 'pri.store.size': '3.9kb'}, {'health': 'green', 'status': 'open', 'index': 'otel-v1-apm-span-sample', 'uuid': 'd8hxfImBTbG9YDzanq

AuthorizationException: AuthorizationException(403, 'security_exception', 'no permissions for [indices:monitor/settings/get] and User [name=arn:aws:iam::169189304433:role/RedcoatExpressRagAdmin, backend_roles=[arn:aws:iam::169189304433:role/RedcoatExpressRagAdmin], requestedTenant=null]')